In [114]:
import json

def process_json_data(json_files):
    """
    여러 JSON 파일을 읽고 데이터를 통합한 후 특정 형식의 문자열 리스트로 반환합니다.

    Parameters:
        json_files (list): JSON 파일 경로 리스트

    Returns:
        list: 파일 데이터에서 'title'과 'content'를 읽어 특정 형식으로 변환한 리스트
    """
    all_json_data = []
    for file_path in json_files:
        try:
            with open(file_path, 'r', encoding='utf-8') as file:
                data = json.load(file)
                all_json_data.extend(data)
        except FileNotFoundError:
            print(f"Error: 파일을 찾을 수 없습니다 - {file_path}")
        except json.JSONDecodeError:
            print(f"Error: JSON 파일 형식이 잘못되었습니다 - {file_path}")

    # 'title'과 'content'를 읽어 특정 형식으로 반환
    return [
        f"Title: {item.get('title', 'N/A')}\nContent: {item.get('content', 'N/A')}"
        for item in all_json_data
    ]

In [115]:
json_files = ['./documents/filtered_unsolved_cases.json', './documents/korea_crime.json']

In [116]:
crimes = process_json_data(json_files)

In [117]:
from llm_module.document_process import *

In [118]:
import os
import openai
from openai import OpenAI
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

openai.api_key = os.environ.get("MY_OPENAI_API_KEY")

In [ ]:
# from langchain.memory import ConversationSummaryMemory

# def summarize_documents(SPLITS):
#     llm = ChatOpenAI(
#                 model="gpt-4o-mini",
#                 api_key=openai.api_key,
#                 max_tokens=500,
#                 temperature=0.0,
#             )
#     summaries = []
#     memory = ConversationSummaryMemory(
#         llm=llm, return_messages=True)
    
#     count = 0
#     for SPLIT in SPLITS:
#         SPLIT = SPLIT.page_content

#         try:
#             context = memory.load_memory_variables({})["history"]
#             prompt = ChatPromptTemplate.from_template(
#                 """
#                 persona : script writer
#                 language : only in korean
#                 research key elements froms docs for script writing
#                 only write about context, write 'N/A if it's not relevant
#                 avoid same content
#                 **hide header like '###'**,
                
#                 <context>
#                 if not, fill it in
#                 {context}
#                 </context>
                
#                 <sample>
#                 # main themes and topics
#                 - what is the central topic of the document
#                 - identify key themes, historical events, or cultural elements

#                 # key figures
#                 - note the names of important people mentioned
#                 - record their roles, achievements, and relevance to the topic

#                 # significant events
#                 - highlight major incidents or milestones related to the topic
#                 - record dates, locations, and outcomes of these events

#                 # background and context
#                 - understand the historical, cultural, or social context of the topic
#                 - identify underlying causes or conditions that shaped the events or subject

#                 # notable locations
#                 - list specific places mentioned in the document
#                 - consider how these locations influence the story

#                 # conflicts and resolutions
#                 - identify key conflicts, challenges, or controversies
#                 - note how they were addressed or resolved

#                 # unique details or anecdotes
#                 - record any interesting or unusual facts that could add depth or intrigue to your story
#                 - look for lesser-known aspects that can provide a fresh perspective
#                 </sample>

#                 <docs>
#                 {SPLIT}
#                 </docs>
#                 """
#             )
#             chain = prompt | llm | StrOutputParser()
#             summary = chain.invoke({"SPLIT": SPLIT, 'context' : context})
#             memory.save_context({"input": f'summary # {count}'}, {"output": summary})
#             summaries.append(summary)
#             count+=1

#         except Exception as e:
#             # 오류 처리: 만약 API 호출 중에 문제가 발생하면 오류 메시지 추가
#             print(f"Error summarizing document: {e}")
#             summaries.append(f"Error summarizing document: {e}")

#     return "".join(summaries), memory

In [138]:
from langchain.memory import ConversationSummaryMemory

def documents_filter(SPLITS):
    llm = ChatOpenAI(
                model="gpt-4o-mini",
                api_key=openai.api_key,
                max_tokens=500,
                temperature=0.0,
            )
    summaries = []
    memory = ConversationSummaryMemory(
        llm=llm, return_messages=True)
    
    count = 0
    for SPLIT in SPLITS:
        SPLIT = SPLIT.page_content

        try:
            context = memory.load_memory_variables({})["history"]
            prompt = ChatPromptTemplate.from_template(
                """
                persona : documents filter
                language : only in korean
                extract the parts related to the context and ignore the rest,
                write blanck if it's not relevant,
                
                <context>
                {context}
                </context>
                
                <docs>
                {SPLIT}
                </docs>
                """
            )
            chain = prompt | llm | StrOutputParser()
            summary = chain.invoke({"SPLIT": SPLIT, 'context' : context})
            memory.save_context({"input": f'summary # {count}'}, {"output": summary})
            summaries.append(summary)
            count+=1

        except Exception as e:
            # 오류 처리: 만약 API 호출 중에 문제가 발생하면 오류 메시지 추가
            print(f"Error summarizing document: {e}")
            summaries.append(f"Error summarizing document: {e}")

    return "".join(summaries), memory

In [132]:
def generate_script(summaries):
    llm = ChatOpenAI(
        model="gpt-4o-mini",
        api_key=openai.api_key,
        max_tokens=4000,
        temperature=0.0,
    )
    prompt = ChatPromptTemplate.from_template(
    """
    persona = script writer
    language = only in korean
    use input,
    refer to sample,
    write about time, character, event,
    write only fact
    ignore the mere listing of facts and write N/A
 
    <sample>
    # title : title of script
    # prologue 1 : song, movie, book, show about subject
    - coontent :
    # prologue 2 : explain about subject
    - coontent :
    # prologue 3 : explain about character
    - coontent :
    # exposition 1 : historical background of subject
    - coontent :
    # exposition 2 : history of character
    - coontent :
    # exposition 3 : beginning of event
    - coontent :
    # development 1 : situation, action, static of character
    - coontent :
    # development 2 : influence of event
    - coontent :
    # development 3 : reaction of people
    - coontent :
    # climax 1 : event and effect bigger
    - coontent :
    # climax 2 : dramatic action, conflict
    - coontent :
    # climax 3 : falling Action
    - coontent :
    # denouement : resolution
    - coontent :
    # epilogue : message, remaining
    - coontent :
    </sample>

    <input>
    {summaries}
    </input>

    """
    )
    chain = prompt | llm | StrOutputParser()
    script = chain.invoke({"summaries": summaries})
    return script


In [133]:
from pprint import pprint
sample = crimes[0]
pprint(sample)

('Title: 정인숙 피살사건\n'
 'Content: 정인숙 피살사건(鄭仁淑 被殺事件)은제3공화국당시인1970년3월 17일밤 11시경,대한민국서울특별시마포구합정동절두산근처 '
 '도로에서 발생한 교통사고를 가장한 총격 살인사건으로 고급 요정 종업원인정인숙이 사망한 사건이다.[1][2]피해자 정인숙(본명: 정금지)은 '
 '총상으로 요절하였고 그의 차를 운전하던 넷째 오빠 정종욱은 넓적다리에 관통상을 입었으나 지나가던 택시기사에게 도움을 청하여 구조되었다.\n'
 '정인숙은 당시 출산한 아이의 아버지인대한민국의 국무총리정일권과 갈등관계에 있었고,신민당은 이 사건의 배후로 정부 고위층의 개입 의혹을 '
 '제기했으나 유야무야 묻혀져 아직까지의문사로 남아있다.\n'
 '경과[편집]\n'
 '경과\n'
 '1970년3월 17일밤 11시경, 서울마포구합정동절두산 부근의강변3로에 멈춰 서 있는검은색 코로나 승용차에서 권총에 넓적다리를 관통당해 '
 '신음하고 있는 한 사내와, 머리와 가슴에 총을 맞아 이미 숨진 한 젊은 여인이 발견되었다. 부상당한 사내는정종욱(당시 34세), 숨진 '
 '여인은 정인숙(당시 25세)으로 두 사람은 남매 관계로 밝혀졌다.\n'
 '당시 26세였던 정인숙에게는 3살 된 아들이 1명 있는 것으로 확인되었다.[3][4]아이의 아버지에 대해서는 당시 정부의 한 유력인사가 '
 '지목되기도 했다.\n'
 '고위 공무원의 딸로 태어나 대학을 중퇴하였으며 여러 명의 친오빠가 있었다. 그 뒤 정인숙은 당시한일회담도 이루어진 선운각 등 최고급 '
 '요정에서 호스티스로 일하고 있었다.[4]\n'
 '나중에 정인숙의 집에서 발견된 정인숙의 소지품에선 정관계 고위층의 명함 26장이 포함된 33장의 명함이 쏟아져 나왔는데, '
 '명단에는박정희,정일권,이후락,김형욱등 대다수 5.16 주체세력들이  포함되어 있었다.[5]이후 경찰 수사는 지지부진해졌고 언론 보도가 '
 '수사를 대신하게 되었다. 언론은 정인숙에게 숨겨진 아들(정일성,정성일, 또는박승일(68년

In [134]:
splits = split_texts(sample)
splits

[Document(metadata={}, page_content='Title: 정인숙 피살사건\nContent: 정인숙 피살사건(鄭仁淑 被殺事件)은제3공화국당시인1970년3월 17일밤 11시경,대한민국서울특별시마포구합정동절두산근처 도로에서 발생한 교통사고를 가장한 총격 살인사건으로 고급 요정 종업원인정인숙이 사망한 사건이다.[1][2]피해자 정인숙(본명: 정금지)은 총상으로 요절하였고 그의 차를 운전하던 넷째 오빠 정종욱은 넓적다리에 관통상을 입었으나 지나가던 택시기사에게 도움을 청하여 구조되었다.\n정인숙은 당시 출산한 아이의 아버지인대한민국의 국무총리정일권과 갈등관계에 있었고,신민당은 이 사건의 배후로 정부 고위층의 개입 의혹을 제기했으나 유야무야 묻혀져 아직까지의문사로 남아있다.\n경과[편집]\n경과\n1970년3월 17일밤 11시경, 서울마포구합정동절두산 부근의강변3로에 멈춰 서 있는검은색 코로나 승용차에서 권총에 넓적다리를 관통당해 신음하고 있는 한 사내와, 머리와 가슴에 총을 맞아 이미 숨진 한 젊은 여인이 발견되었다. 부상당한 사내는정종욱(당시 34세), 숨진 여인은 정인숙(당시 25세)으로 두 사람은 남매 관계로 밝혀졌다.\n당시 26세였던 정인숙에게는 3살 된 아들이 1명 있는 것으로 확인되었다.[3][4]아이의 아버지에 대해서는 당시 정부의 한 유력인사가 지목되기도 했다.\n고위 공무원의 딸로 태어나 대학을 중퇴하였으며 여러 명의 친오빠가 있었다. 그 뒤 정인숙은 당시한일회담도 이루어진 선운각 등 최고급 요정에서 호스티스로 일하고 있었다.[4]'),
 Document(metadata={}, page_content="당시 26세였던 정인숙에게는 3살 된 아들이 1명 있는 것으로 확인되었다.[3][4]아이의 아버지에 대해서는 당시 정부의 한 유력인사가 지목되기도 했다.\n고위 공무원의 딸로 태어나 대학을 중퇴하였으며 여러 명의 친오빠가 있었다. 그 뒤 정인숙은 당시한일회담도 이루어진 선운각 등 최고급 요정에서 호스티스로 일하고 있었다.[4]\n나

In [139]:
summary, memory = documents_filter(splits)

In [140]:
pprint(summary)

('<context>\n'
 'N/A\n'
 '</context>\n'
 '\n'
 '<docs>\n'
 'Title: 정인숙 피살사건\n'
 'Content: 정인숙 피살사건(鄭仁淑 被殺事件)은 제3공화국 당시인 1970년 3월 17일 밤 11시경, 대한민국 서울특별시 마포구 '
 '합정동 절두산 근처 도로에서 발생한 교통사고를 가장한 총격 살인사건으로 고급 요정 종업원인 정인숙이 사망한 사건이다. 피해자 '
 '정인숙(본명: 정금지)은 총상으로 요절하였고 그의 차를 운전하던 넷째 오빠 정종욱은 넓적다리에 관통상을 입었으나 지나가던 택시기사에게 '
 '도움을 청하여 구조되었다. 정인숙은 당시 출산한 아이의 아버지인 대한민국의 국무총리 정일권과 갈등관계에 있었고, 신민당은 이 사건의 '
 '배후로 정부 고위층의 개입 의혹을 제기했으나 유야무야 묻혀져 아직까지 의문사로 남아있다. 1970년 3월 17일 밤 11시경, 서울 '
 '마포구 합정동 절두산 부근의 강변 3로에 멈춰 서 있는 검은색 코로나 승용차에서 권총에 넓적다리를 관통당해 신음하고 있는 한 사내와, '
 '머리와 가슴에 총을 맞아 이미 숨진 한 젊은 여인이 발견되었다. 부상당한 사내는 정종욱(당시 34세), 숨진 여인은 정인숙(당시 '
 '25세)으로 두 사람은 남매 관계로 밝혀졌다. 당시 26세였던 정인숙에게는 3살 된 아들이 1명 있는 것으로 확인되었다. 아이의 아버지에 '
 '대해서는 당시 정부의 한 유력인사가 지목되기도 했다. 고위 공무원의 딸로 태어나 대학을 중퇴하였으며 여러 명의 친오빠가 있었다. 그 뒤 '
 '정인숙은 당시 한일회담도 이루어진 선운각 등 최고급 요정에서 호스티스로 일하고 있었다.\n'
 '</docs정인숙 살해 사건은 1970년 3월 17일 서울에서 발생하였으며, 정인숙은 고급 호스티스로 일하던 중 총에 맞아 사망하였다. '
 '사건은 교통사고로 위장되었고, 그녀의 오빠 정종욱이 부상당했으나 생존하였다. 사건은 정부의 개입 의혹이 제기되었고, 정인숙은 당시 정일권 '
 '

In [142]:
len(summary)

7117

In [143]:
script = generate_script(summary)

In [144]:
pprint(script)

('# title : 정인숙 피살사건\n'
 '# prologue 1 : 사건을 다룬 다큐멘터리\n'
 '- content : 2010년 SBS TV 프로그램에서 정인숙 사건의 실체를 추적하는 내용이 방송되었다.\n'
 '\n'
 '# prologue 2 : 사건 개요\n'
 '- content : 정인숙 피살사건은 1970년 3월 17일 서울에서 발생한 교통사고로 위장된 총격 사건으로, 고급 호스티스인 정인숙이 '
 '사망한 사건이다.\n'
 '\n'
 '# prologue 3 : 정인숙의 배경\n'
 '- content : 정인숙은 고위 공무원의 딸로 태어나 대학을 중퇴하고, 당시 한일회담이 이루어진 선운각 등에서 호스티스로 일하고 '
 '있었다.\n'
 '\n'
 '# exposition 1 : 사건 발생 배경\n'
 '- content : 사건은 제3공화국 시절, 정치적 불안정과 권력자들의 부패가 만연하던 시기에 발생하였다.\n'
 '\n'
 '# exposition 2 : 정인숙의 개인사\n'
 '- content : 정인숙은 26세로, 3세 아들이 있었으며, 아들의 아버지는 당시 정부의 유력 인사로 지목되었다.\n'
 '\n'
 '# exposition 3 : 사건의 시작\n'
 '- content : 1970년 3월 17일 밤 11시경, 서울 마포구 합정동에서 정인숙과 그의 오빠 정종욱이 탑승한 차량이 멈춰 서 '
 '있었고, 이때 총격이 발생하였다.\n'
 '\n'
 '# development 1 : 사건의 상황\n'
 '- content : 정인숙은 총에 맞아 사망하였고, 정종욱은 부상을 입었으나 지나가던 택시기사의 도움으로 구조되었다.\n'
 '\n'
 '# development 2 : 사건의 영향\n'
 '- content : 사건은 정부 고위층의 개입 의혹을 불러일으켰고, 언론은 사건을 보도하며 사회적 파장을 일으켰다.\n'
 '\n'
 '# development 3 : 대중의 반응\n'
 '- content : 사건에 대한 대중의 관심이